In [1]:
import os

project_folder = './pytorch-peds'
os.makedirs(project_folder, exist_ok=True)

In [2]:
import shutil

shutil.copy('script.py', project_folder)

'./pytorch-peds/script.py'

In [1]:
#import sys
#sys.path.insert(0,'./')
# Install pycocotools
# !git clone https://github.com/cocodataset/cocoapi.git
# !ls
# %cd cocoapi/PythonAPI
# !ls
# !pip install cython
# !pip install numpy
# !python setup.py build_ext install

fatal: destination path 'cocoapi' already exists and is not an empty directory.
PyTorch-object-detection-1.ipynb cocoapi
build                            config.json
/Users/gkv/Code/PT-Azure-maskrcnn/cocoapi/PythonAPI
Makefile             dist                 setup.py
PennFudanPed         engine.py            test.zip
__pycache__          pycocoDemo.ipynb     transforms.py
build                pycocoEvalDemo.ipynb utils.py
coco_eval.py         pycocotools          vision
coco_utils.py        pycocotools.egg-info
cocoapi              script.py
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
running install
running bdist_egg
running egg_info
writing pycocotools.egg-info/PKG-INFO
writing dependency_links to pycocotools.egg-info/dependency_links.txt
writing requirements to pycocotools.egg-info/requires.txt
writing top-level names to pycocotools.egg-info/top_level.txt
reading manifest file 'pycocotools.egg-info/SOURCES.txt'
writing manifest file 'pycocotools.e

In [3]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.85


In [4]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [6]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: gopalv-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: aifxdemo


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-02-16T23:53:03.831000+00:00', 'errors': None, 'creationTime': '2020-02-05T18:19:06.976503+00:00', 'modifiedTime': '2020-02-05T18:20:28.120151+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 2, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_NC6'}


In [8]:
import urllib.request
 
url = 'https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip'
 
urllib.request.urlretrieve(url, './test.zip')

('./test.zip', <http.client.HTTPMessage at 0x10cdfdaf0>)

In [9]:
from zipfile import ZipFile

zip = ZipFile(file='./test.zip')
zip.extractall()

In [10]:
!ls PennFudanPed/

Annotation            PedMasks              readme.txt
PNGImages             added-object-list.txt


In [11]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

workspaceblobstore AzureBlob gopalvws3790775563 azureml-blobstore-e47496c6-9688-4277-a05b-ceb722514b9d


In [12]:
ds.upload('./PennFudanPed', target_path='data', overwrite=False)

Uploading an estimated of 512 files
Target already exists. Skipping upload for data/added-object-list.txt
Target already exists. Skipping upload for data/readme.txt
Target already exists. Skipping upload for data/PedMasks/PennPed00048_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00049_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00072_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00073_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00001_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00093_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00092_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00076_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00077_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00005_mask.png
Target already exists. Skipping upload for

Target already exists. Skipping upload for data/PedMasks/FudanPed00059_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00062_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00063_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00032_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00033_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00008_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00009_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00041_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00040_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00036_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00037_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00045_mask.png
Target already exists. Skipping upload for data/

Target already exists. Skipping upload for data/Annotation/FudanPed00063.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00062.txt
Target already exists. Skipping upload for data/Annotation/PennPed00059.txt
Target already exists. Skipping upload for data/Annotation/PennPed00065.txt
Target already exists. Skipping upload for data/Annotation/PennPed00071.txt
Target already exists. Skipping upload for data/Annotation/PennPed00067.txt
Target already exists. Skipping upload for data/Annotation/PennPed00073.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00074.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00060.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00048.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00049.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00061.txt
Target already exists. Skipping upload for data/Annotation/PennPed00072.txt
Targe

Target already exists. Skipping upload for data/Annotation/PennPed00008.txt
Target already exists. Skipping upload for data/Annotation/PennPed00020.txt
Target already exists. Skipping upload for data/Annotation/PennPed00034.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00033.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00027.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00026.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00032.txt
Target already exists. Skipping upload for data/Annotation/PennPed00035.txt
Target already exists. Skipping upload for data/Annotation/PennPed00021.txt
Target already exists. Skipping upload for data/Annotation/PennPed00009.txt
Target already exists. Skipping upload for data/Annotation/PennPed00037.txt
Target already exists. Skipping upload for data/Annotation/PennPed00023.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00018.txt
Target 

Target already exists. Skipping upload for data/PNGImages/PennPed00024.png
Target already exists. Skipping upload for data/PNGImages/PennPed00030.png
Target already exists. Skipping upload for data/PNGImages/PennPed00031.png
Target already exists. Skipping upload for data/PNGImages/PennPed00025.png
Target already exists. Skipping upload for data/PNGImages/PennPed00019.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00022.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00036.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00032.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00026.png
Target already exists. Skipping upload for data/PNGImages/PennPed00021.png
Target already exists. Skipping upload for data/PNGImages/PennPed00035.png
Target already exists. Skipping upload for data/PNGImages/PennPed00009.png
Target already exists. Skipping upload for data/PNGImages/PennPed00008.png
Target already exists

$AZUREML_DATAREFERENCE_26d2302af8f64b43a3c0bc9a249bffab

In [13]:
from azureml.core import Dataset
# create a FileDataset pointing to files in 'animals' folder and its subfolders recursively
datastore_paths = [(ds, 'data')]
penn_ds = Dataset.File.from_files(path=datastore_paths)
penn_ds.register(workspace=ws,
                 name='penn_ds',
                 description='Penn Fudan pedestrian data')

{
  "source": [
    "('workspaceblobstore', 'data')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "1cd2dd50-ae6d-44f9-81e2-f0044c862efe",
    "name": "penn_ds",
    "version": 1,
    "description": "Penn Fudan pedestrian data",
    "workspace": "Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo')"
  }
}

In [13]:
# dataset_name = 'penn_ds'

# # Get a dataset by name
# penn1_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [17]:
# from PIL import Image
# import os
# import glob


# with penn_ds.mount() as mount_context:
#     # list top level mounted files and folders in the dataset
#     imgs_path =(os.path.join(mount_context.mount_point, 'PNGImages'))
#     print(f'imgs_path is {imgs_path}')
#     print(list(sorted(os.listdir(imgs_path))))
#     Image.open(os.path.join(mount_context.mount_point, 'PNGImages/PennPed00036.png'))
penn1_ds.as_named_input('test').as_mount()

In [15]:
# PennFudanDataset(penn_ds)

In [22]:
!ls
%cd ..
!ls

LICENSE     README.rst  hubconf.py  setup.cfg   test        tox.ini
MANIFEST.in docs        references  setup.py    torchvision
/Users/gkv/MS/pytorch-object
PennFudanPed                     pytorch-peds
PyTorch-object-detection-1.ipynb script.py
coco_eval.py                     test.zip
coco_utils.py                    transforms.py
config.json                      utils.py
engine.py                        vision


In [24]:
!git clone https://github.com/pytorch/vision.git

!git checkout v0.3.0

%cd vision
!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../
%cd ..

fatal: destination path 'vision' already exists and is not an empty directory.
error: pathspec 'v0.3.0' did not match any file(s) known to git
/Users/gkv/MS/pytorch-object/vision
/Users/gkv/MS/pytorch-object


In [26]:
import shutil

os.listdir()

files_to_copy = ['utils', 'transforms', 'coco_eval', 'engine', 'coco_utils']
for file in files_to_copy:
    shutil.copy('./vision/references/detection/'+ file + '.py', project_folder)


In [27]:
from azureml.core import Experiment

experiment_name = 'pytorch-peds'
experiment = Experiment(ws, name=experiment_name)

In [56]:
from azureml.core import Environment

my_env = Environment(name='maskr-docker')
my_env.docker.enabled = True
with open("Dockerfile1", "r") as f:
    dockerfile_contents_of_your_base_image=f.read()
my_env.docker.base_dockerfile=dockerfile_contents_of_your_base_image 

In [57]:
from azureml.train.estimator import Estimator

my_est = Estimator(
    source_directory=project_folder,
    entry_script="script.py",
    compute_target="local",
    use_gpu=True,
    conda_packages=['torchvision">=0.5.0"', 'torch', 'numpy', 'cython'])

SyntaxError: invalid syntax (<ipython-input-57-cd53a43bd251>, line 8)

In [53]:
from azureml.train.dnn import PyTorch
from azureml.core import ScriptRunConfig

# follow pattern from here: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#use-environments-for-training

# Add training script to run config
runconfig = ScriptRunConfig(source_directory=project_folder, script="script.py")

# Attach compute target to run config
runconfig.run_config.target = "local"

my_env.docker.base_image = None
# Attach environment to run config
runconfig.run_config.environment = my_env

# Submit run 
run = experiment.submit(runconfig)
print(run)

Run(Experiment: pytorch-peds,
Id: pytorch-peds_1582505053_9447115a,
Type: azureml.scriptrun,
Status: Starting)


In [54]:
# to get more details of your run
print(run.get_details())

{'runId': 'pytorch-peds_1582505053_9447115a', 'target': 'local', 'status': 'Starting', 'properties': {'_azureml.ComputeTargetType': 'local', 'ContentSnapshotId': '02c14d39-11b2-4acf-8b98-43ed768628b0', 'azureml.git.repository_uri': 'https://github.com/gvashishtha/pytorch-object.git', 'mlflow.source.git.repoURL': 'https://github.com/gvashishtha/pytorch-object.git', 'azureml.git.branch': 'docker', 'mlflow.source.git.branch': 'docker', 'azureml.git.commit': '25176223476f74c2f677238602d941dc15c460a7', 'mlflow.source.git.commit': '25176223476f74c2f677238602d941dc15c460a7', 'azureml.git.dirty': 'False'}, 'inputDatasets': [], 'runDefinition': {'script': 'script.py', 'useAbsolutePath': False, 'arguments': [], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'local', 'dataReferences': {}, 'data': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'environment': {'name': 'maskr-docker', 'version': 'Autosave_2020-02-24T00:44:14Z_31b847fd'

In [55]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1582505053_9447115a
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1582505053_9447115a?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/aifxdemo/workspaces/gopalv-ws

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 71093
Running: ['/bin/bash', '/private/var/folders/zq/sdr9gtc92pd15gf53xb1tct80000gn/T/azureml_runs/pytorch-peds_1582505053_9447115a/azureml-environment-setup/docker_env_checker.sh']

Materialized image not found on target: azureml/azureml_1f4d73b4a8e90d6f4ce5fe3e847ece69


Logging experiment preparation status in history service.
Running: ['/bin/bash', '/private/var/folders/zq/sdr9gtc92pd15gf53xb1tct80000gn/T/azureml_runs/pytorch-peds_1582505053_9447115a/azureml-environment-setup/docker_env_builder.sh']
Running: ['docker', 'build', '-f', 'azureml-environment-setup/Dockerfile', '-

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/miniconda

  added / updated specs:
    - python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2019.11.28         |           py37_0         153 KB
    conda-4.8.2                |           py37_0         2.8 MB
    openssl-1.1.1d             |       h7b6447c_4         2.5 MB
    ------------------------------------------------------------
                                           Total:         5.6 MB

The following packages will be UPDATED:

  ca-certificates                              2019.10.16-0 --> 2020.1.1-0
  certifi                                  2019.9.11-py37_0 --> 2019.11.28-py37_0
  conda                                       4.7.12-py37_0 --> 4.8.2-py37_0
  openssl                                 1.1.1d-

WARNING - Retrying (Retry(total=2, connect=3, read=3, redirect=None, status=None)) after connection broken by 'SSLError(SSLError("bad handshake: SysCallError(54, 'ECONNRESET')"))': /history/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs/pytorch-peds_1582505053_9447115a/details


pytorch-1.4.0        | 432.9 MB  | ########## | 100% 
magma-cuda100-2.5.1  | 52.8 MB   | ########## | 100% 
jpeg-9b              | 214 KB    | ########## | 100% 
libtiff-4.1.0        | 447 KB    | ########## | 100% 
libpng-1.6.37        | 278 KB    | ########## | 100% 
torchvision-0.5.0    | 9.1 MB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Cache location: /opt/miniconda/pkgs
Will remove the following tarballs:

/opt/miniconda/pkgs
-------------------
openssl-1.1.1d-h7b6447c_4.conda              2.5 MB
olefile-0.46-py37_0.conda                     50 KB
jpeg-9b-h024ee3a_2.conda                     214 KB
cudatoolkit-10.1.243-h6bb024c_0.conda      347.4 MB
pytorch-1.4.0-py3.7_cuda10.1.243_cudnn7.6.3_0.tar.bz2   432.9 MB
libgfortran-ng-7.3.0-hdf63c60_0.conda       1006 KB
torchvision-0.5.0-py37_cu101.tar.bz2         9.1 MB
ninja-1.9.0-py37hfd86e86_0.conda             1.2 MB
pillow-7

...

libtool: install: /usr/bin/install -c .libs/mca_atomic_basic.soT /usr/local/lib/openmpi/mca_atomic_basic.so
libtool: install: /usr/bin/install -c .libs/mca_atomic_basic.lai /usr/local/lib/openmpi/mca_atomic_basic.la
libtool: finish: PATH="/opt/miniconda/bin:/opt/miniconda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/sbin" ldconfig -n /usr/local/lib/openmpi
----------------------------------------------------------------------
Libraries have been installed in:
   /usr/local/lib/openmpi

If you ever happen to want to link against installed libraries
in a given directory, LIBDIR, you must either use libtool, and
specify the full pathname of the library, or use the '-LLIBDIR'
flag during linking and do at least one of the following:
   - add LIBDIR to the 'LD_LIBRARY_PATH' environment variable
     during execution
   - add LIBDIR to the 'LD_RUN_PATH' environment variable
     during linking
   - use the '-Wl,-rpath -Wl,LIBDIR'

Removing intermediate container 52092559506e
 ---> 76e36d49fbdf
Step 24/38 : RUN git clone https://github.com/cocodataset/cocoapi.git && cd cocoapi/PythonAPI && python setup.py build_ext install
 ---> Running in 0e84fa332c57
Cloning into 'cocoapi'...
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
gcc -pthread -B /opt/miniconda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/miniconda/lib/python3.7/site-packages/numpy/core/include -I../common -I/opt/miniconda/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function â€˜rleDecodeâ€™:
../common/maskApi.c:46:7: warning: this â€˜forâ€™ clause does not guard... [-Wmisleading-indentation]
   

Best match: matplotlib 3.2.0rc3
Processing matplotlib-3.2.0rc3-cp37-cp37m-manylinux1_x86_64.whl
Installing matplotlib-3.2.0rc3-cp37-cp37m-manylinux1_x86_64.whl to /opt/miniconda/lib/python3.7/site-packages
writing requirements to /opt/miniconda/lib/python3.7/site-packages/matplotlib-3.2.0rc3-py3.7-linux-x86_64.egg/EGG-INFO/requires.txt
Adding matplotlib 3.2.0rc3 to easy-install.pth file

Installed /opt/miniconda/lib/python3.7/site-packages/matplotlib-3.2.0rc3-py3.7-linux-x86_64.egg
Searching for python-dateutil>=2.1
Reading https://pypi.org/simple/python-dateutil/
Best match: python-dateutil 2.8.1
Processing python_dateutil-2.8.1-py2.py3-none-any.whl
Installing python_dateutil-2.8.1-py2.py3-none-any.whl to /opt/miniconda/lib/python3.7/site-packages
writing requirements to /opt/miniconda/lib/python3.7/site-packages/python_dateutil-2.8.1-py3.7.egg/EGG-INFO/requires.txt
Adding python-dateutil 2.8.1 to easy-install.pth file

Installed /opt/miniconda/lib/python3.7/site-packages/python_dateu


Streaming azureml-logs/70_driver_log.txt

Starting the daemon thread to refresh tokens in background for process with pid = 7
Entering Run History Context Manager.
Preparing to call script [ script.py ] with arguments: []
After variable expansion, calling script [ script.py ] with arguments: []



The experiment failed. Finalizing run...
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 7
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.002708911895751953 seconds
Traceback (most recent call last):
  File "script.py", line 11, in <module>
    from engine import train_one_epoch, evaluate
  File "/azureml-run/engine.py", line 4, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


Execution Summary
RunId: pytorch-peds_1582505053_9447115a
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1582505053_94

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with ModuleNotFoundError: No module named 'torch'",
        "detailsUri": "https://aka.ms/azureml-known-errors",
        "details": [],
        "debugInfo": {
            "type": "ModuleNotFoundError",
            "message": "No module named 'torch'",
            "stackTrace": "  File \"azureml-setup/context_manager_injector.py\", line 127, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_1b417bb747e35859ebf611fb43071e9c/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azureml_1b417bb747e35859ebf611fb43071e9c/lib/python3.6/runpy.py\", line 96, in _run_module_code\n    mod_name, mod_spec, pkg_name, script_name)\n  File \"/azureml-envs/azureml_1b417bb747e35859ebf611fb43071e9c/lib/python3.6/

In [62]:
from azureml.train.dnn import PyTorch

script_params = {
    '--num_epochs': 10,
    '--output_dir': './outputs'
}

estimator = PyTorch(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='script.py',
                    use_gpu=True,
                    framework_version=1.4,
                    conda_packages=['cocoapi','torchvision">=0.5.0"'])

TrainingException: TrainingException:
	Message: PyTorch 1.4 is not currently supported by the PyTorch estimator. Check https://docs.microsoft.com/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py for all supported versions. To use PyTorch 1.4, switch to the generic Estimator class for your experiment.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "PyTorch 1.4 is not currently supported by the PyTorch estimator. Check https://docs.microsoft.com/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py for all supported versions. To use PyTorch 1.4, switch to the generic Estimator class for your experiment."
    }
}

In [59]:
run = experiment.submit(estimator)
print(run.get_details())

{'runId': 'pytorch-peds_1582506530_b7f7b7ba', 'target': 'gpu-cluster', 'status': 'Starting', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '02c14d39-11b2-4acf-8b98-43ed768628b0', 'azureml.git.repository_uri': 'https://github.com/gvashishtha/pytorch-object.git', 'mlflow.source.git.repoURL': 'https://github.com/gvashishtha/pytorch-object.git', 'azureml.git.branch': 'docker', 'mlflow.source.git.branch': 'docker', 'azureml.git.commit': '25176223476f74c2f677238602d941dc15c460a7', 'mlflow.source.git.commit': '25176223476f74c2f677238602d941dc15c460a7', 'azureml.git.dirty': 'False'}, 'inputDatasets': [], 'runDefinition': {'script': 'script.py', 'useAbsolutePath': False, 'arguments': ['--num_epochs', '10', '--output_dir', './outputs'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'gpu-cluster', 'dataReferences': {}, 'data': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'environment': {'name': '

In [60]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1582506530_b7f7b7ba
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1582506530_b7f7b7ba?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/aifxdemo/workspaces/gopalv-ws

Streaming azureml-logs/20_image_build_log.txt

2020/02/24 01:09:10 Downloading source code...
2020/02/24 01:09:12 Finished downloading source code
2020/02/24 01:09:12 Creating Docker network: acb_default_network, driver: 'bridge'
2020/02/24 01:09:12 Successfully set up Docker network: acb_default_network
2020/02/24 01:09:12 Setting up Docker configuration...
2020/02/24 01:09:13 Successfully set up Docker configuration
2020/02/24 01:09:13 Logging in to registry: gopalvwsdebb3b12.azurecr.io
2020/02/24 01:09:14 Successfully logged into gopalvwsdebb3b12.azurecr.io
2020/02/24 01:09:14 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/02/24 01:09:14 Scanning for dependencies...
2020/02/24 01:09:15 Su

Executing transaction: ...working... done
ERROR: Could not find a version that satisfies the requirement cocoapi (from -r /azureml-environment-setup/condaenv.65gvb5se.requirements.txt (line 1)) (from versions: none)
ERROR: No matching distribution found for cocoapi (from -r /azureml-environment-setup/condaenv.65gvb5se.requirements.txt (line 1))


CondaValueError: pip returned an error

The command '/bin/sh -c ldconfig /usr/local/cuda/lib64/stubs && conda env create -p /azureml-envs/azureml_5ad52abb83b25185b008ba0673cb9abe -f azureml-environment-setup/mutated_conda_dependencies.yml && rm -rf "$HOME/.cache/pip" && conda clean -aqy && CONDA_ROOT_DIR=$(conda info --root) && rm -rf "$CONDA_ROOT_DIR/pkgs" && find "$CONDA_ROOT_DIR" -type d -name __pycache__ -exec rm -rf {} + && ldconfig' returned a non-zero code: 1
2020/02/24 01:11:43 Container failed during run: acb_step_0. No retries remaining.
failed to run step ID: acb_step_0: exit status 1

Run ID: ccg failed after 2m33s. Error: failed d

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt",
        "details": []
    },
    "correlation": {
        "operation": null,
        "request": "ba9ee0061c51d5a1"
    },
    "environment": "westus2",
    "location": "westus2",
    "time": "2020-02-24T01:11:48.710982Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt\",\n        \"details\": []\n    },\n    \"correlation\": {\n        \"operation\": null,\n        \"request\": \"ba9ee0061c51d5a1\"\n    },\n    \"environment\": \"westus2\",\n    \"location\": \"westus2\",\n    \"time\": \"2020-02-24T01:11:48.710982Z\"\n}"
    }
}

In [14]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        with self.dataset.mount() as mount_context:
            self.imgs = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PNGImages"))))
            self.masks = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        with self.dataset.mount() as mount_context:
            img_path = os.path.join(mount_context.mount_point, "PNGImages", self.imgs[idx])
            mask_path = os.path.join(mount_context.mount_point, "PedMasks", self.masks[idx])
            img = Image.open(img_path).convert("RGB")
            # note that we haven't converted the mask to RGB,
            # because each color corresponds to a different instance
            # with 0 being background
            mask = Image.open(mask_path)

            mask = np.array(mask)
            # instances are encoded as different colors
            obj_ids = np.unique(mask)
            # first id is the background, so remove it
            obj_ids = obj_ids[1:]

            # split the color-encoded mask into a set
            # of binary masks
            masks = mask == obj_ids[:, None, None]

            # get bounding box coordinates for each mask
            num_objs = len(obj_ids)
            boxes = []
            for i in range(num_objs):
                pos = np.where(masks[i])
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                boxes.append([xmin, ymin, xmax, ymax])

            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            # there is only one class
            labels = torch.ones((num_objs,), dtype=torch.int64)
            masks = torch.as_tensor(masks, dtype=torch.uint8)

            image_id = torch.tensor([idx])
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            # suppose all instances are not crowd
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

            target = {}
            target["boxes"] = boxes
            target["labels"] = labels
            target["masks"] = masks
            target["image_id"] = image_id
            target["area"] = area
            target["iscrowd"] = iscrowd

            if self.transforms is not None:
                img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [16]:
!pip install --upgrade torchvision

Requirement already up-to-date: torchvision in /Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages (0.5.0)


In [27]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [19]:
import sys
sys.path.insert(0,'./')

In [20]:
from platform import python_version

print(python_version())
help("modules")

3.8.1

Please wait a moment while I gather a list of all available modules...



/Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages/IPython/kernel/__init__.py:12: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  warn("The `IPython.kernel` package has been deprecated since IPython 4.0."
/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:107: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, info.name+'.', onerror)


Cython              asynchat            inspect             random
IPython             asyncio             io                  re
OpenSSL             asyncore            ipaddress           readline
PIL                 atexit              ipykernel           reprlib
PyQt5               attr                ipykernel_launcher  requests
__future__          audioop             ipython_genutils    requests_oauthlib
_abc                automl              ipywidgets          resource
_ast                autoreload          isodate             rlcompleter
_asyncio            azureml             itertools           rmagic
_bisect             backcall            jaraco              runpy
_blake2             backports           jedi                sched
_bootlocale         base64              jeepney             secrets
_bz2                bdb                 jinja2              secretstorage
_cffi_backend       binascii            jmespath            select
_codecs             binhex           

/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:92: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  __import__(info.name)


In [28]:
# from engine import train_one_epoch, evaluate
# import utils
# import transforms as T

# def get_transform(train):
#     transforms = []
#     # converts the image, a PIL image, into a PyTorch Tensor
#     transforms.append(T.ToTensor())
#     if train:
#         # during training, randomly flip the training images
#         # and ground-truth for data augmentation
#         transforms.append(T.RandomHorizontalFlip(0.5))
#     return T.Compose(transforms)


In [33]:
# # use our dataset and defined transformations
# dataset = PennFudanDataset(penn_ds, get_transform(train=True))
# dataset_test = PennFudanDataset(penn_ds, get_transform(train=False))

# # split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# # define training and validation data loaders
# data_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=2, shuffle=True, num_workers=4,
#     collate_fn=utils.collate_fn)

# data_loader_test = torch.utils.data.DataLoader(
#     dataset_test, batch_size=1, shuffle=False, num_workers=4,
#     collate_fn=utils.collate_fn)

In [34]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # our dataset has two classes only - background and person
# num_classes = 2

# # get the model using our helper function
# model = get_instance_segmentation_model(num_classes)
# # move model to the right device
# model.to(device)

# # construct an optimizer
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)

# # and a learning rate scheduler which decreases the learning rate by
# # 10x every 3 epochs
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                                step_size=3,
#                                                gamma=0.1)

In [1]:
# num_epochs = 10

# for epoch in range(num_epochs):
#     # train for one epoch, printing every 10 iterations
#     train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
#     # update the learning rate
#     lr_scheduler.step()
#     # evaluate on the test dataset
#     evaluate(model, data_loader_test, device=device)

NameError: name 'train_one_epoch' is not defined

In [ ]:
# # pick one image from the test set
# img, _ = dataset_test[0]
# # put the model in evaluation mode
# model.eval()
# with torch.no_grad():
#     prediction = model([img.to(device)])

In [ ]:
# Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())